## datacheck

In [1]:
import os
import chardet
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def detect_encoding(file_path):
    try:
        with open(file_path, 'rb') as f:
            result = chardet.detect(f.read(1024))
        return [file_path, result.get('encoding')]
    except Exception as e:
        print(f"{file_path} generated an exception: {e}")
        return None

def check_file_encodings(folder_path):
    encoding_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for root, dirs, files in os.walk(folder_path):
            for filename in files:
                if filename.endswith(".txt"):
                    file_path = os.path.join(root, filename)
                    futures.append(executor.submit(detect_encoding, file_path))
        for future in tqdm(futures, desc="Checking encodings"):
            result = future.result()
            if result:
                encoding_data.append(result)
    df = pd.DataFrame(encoding_data, columns=['File Name', 'Encoding Type'])
    return df

# Initialize or load existing summary DataFrame
try:
    df_summary = pd.read_csv('txt_encoding.csv')
except FileNotFoundError:
    df_summary = pd.DataFrame(columns=['File Name', 'Encoding Type'])


# Get all directory names under "/app/data/jrdb_txt/"
list_file_name = [name for name in os.listdir("/app/data/jrdb_txt/") if os.path.isdir(os.path.join("/app/data/jrdb_txt/", name))]

for file_name in list_file_name:
    print(f"Processing {file_name}...")
    folder_path = f"/app/data/jrdb_txt/{file_name}"
    df = check_file_encodings(folder_path)
    unique_encodings = df['Encoding Type'].unique()

    new_row = pd.DataFrame({'File Name': [file_name], 'Encoding Type': [unique_encodings]})
    df_summary = pd.concat([df_summary, new_row], ignore_index=True)

    df_summary.to_csv('txt_encoding.csv', index=False)

print("Finished processing all directories.")


Processing BAC...


Checking encodings:  58%|█████▊    | 1544/2655 [00:01<00:01, 813.41it/s] 


KeyboardInterrupt: 

In [4]:
df_summary


,File Name,Encoding Type
0,BAC,['SHIFT_JIS']
1,BAC,[SHIFT_JIS]
2,CHA,"[MacRoman, Johab, Windows-1252]"
3,CYB,"[ascii, MacRoman]"
4,CZA,[SHIFT_JIS]
5,JOA,[SHIFT_JIS]
6,KAB,"[Windows-1252, None, MacRoman, IBM866]"
7,KKA,[ascii]
8,KYI,"[CP932, SHIFT_JIS]"
9,KZA,[SHIFT_JIS]


## type_check_single

In [2]:
import os
from pathlib import Path
import chardet  # エンコーディングを自動判別するためのライブラリ

def check_encoding(base_directory):
    base_directory_path = Path(base_directory)
    years = [f.name for f in base_directory_path.iterdir() if f.is_dir()]

    for year in years:
        directory_path = base_directory_path / year
        for filename in os.listdir(directory_path):
            if filename.endswith(".txt"):
                file_path = directory_path / filename
                with open(file_path, 'rb') as f:
                    result = chardet.detect(f.read())
                if result['encoding'] != 'SHIFT_JIS':
                    print(f"File {file_path} is not encoded in SHIFT_JIS. Detected encoding: {result['encoding']}")


## Encoding

### BAC

In [4]:
File_type = 'BAC'
year = 2022
df_a = pd.read_csv(f'/app/data/jrdb_csv/{File_type}/{File_type}_{year}.csv')
df_a
df_a['日'].unique()


array(['1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c'],
      dtype=object)

In [4]:
# # df_bac[df_bac['回数']=='40111000']
# # df_bac[df_bac['場コード']=='08']
# print(df_bac.columns)
# # print(df_bac.dtypes)
# print(df_bac['グレード'].unique())
# # df_bac.loc[1, 'レース名']

# # df_bac[['３着賞金']]


### CHA

In [7]:
File_type = 'CHA'
year = 2023
df_a = pd.read_csv(f'/app/data/jrdb_csv/{File_type}/{File_type}_{year}.csv')
df_a


/tmp/ipykernel_559/1966425011.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_a = pd.read_csv(f'/app/data/jrdb_csv/{File_type}/{File_type}_{year}.csv')


,場コード,年,回,日,Ｒ,馬番,曜日,調教年月日,回数,調教コースコード,追切種類,追い状態,乗り役,調教Ｆ,テンＦ,中間Ｆ,終いＦ,テンＦ指数,中間Ｆ指数,終いＦ指数,追切指数,併せ結果,追切種類（併せ馬）,年齢,クラス,予備
0,6,23,1,1,1,1,日,20221225,1,02,1.0,4.0,1.0,6,158.0,134.0,121.0,1.0,11.0,21.0,41.0,2.0,3.0,5.0,10,NaN
1,6,23,1,1,1,2,水,20221228,0,01,3.0,2.0,5.0,4,146.0,135.0,126.0,19.0,12.0,21.0,52.0,2.0,2.0,2.0,A3,NaN
2,6,23,1,1,1,3,土,20221231,1,02,2.0,9.0,1.0,6,163.0,136.0,120.0,0.0,13.0,26.0,48.0,3.0,2.0,3.0,10,NaN
3,6,23,1,1,1,4,火,20230103,1,02,1.0,4.0,1.0,6,148.0,138.0,130.0,14.0,11.0,15.0,40.0,3.0,3.0,30.0,NaN,NaN
4,6,23,1,1,1,5,火,20230103,1,02,3.0,2.0,4.0,6,167.0,134.0,119.0,0.0,14.0,24.0,48.0,2.0,NaN,3.0,A1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38014,8,23,2,5,12,10,月,20231009,0,11,3.0,2.0,1.0,4,139.0,127.0,125.0,29.0,27.0,26.0,81.0,NaN,NaN,NaN,NaN,NaN
38015,8,23,2,5,12,11,木,20231012,0,11,1.0,4.0,1.0,4,136.0,130.0,132.0,30.0,20.0,15.0,59.0,NaN,NaN,NaN,NaN,NaN
38016,8,23,2,5,12,12,木,20231012,0,11,3.0,2.0,1.0,4,153.0,137.0,129.0,13.0,9.0,18.0,42.0,3.0,3.0,4.0,OP,NaN
38017,8,23,2,5,12,13,木,20231012,0,11,3.0,2.0,1.0,4,152.0,132.0,122.0,14.0,18.0,25.0,61.0,NaN,NaN,NaN,NaN,NaN


In [5]:
# エンコーディングを手動で確認するためのスクリプト
def check_encoding_manually(file_path):
    encodings_to_try = ['utf-8',  'euc-jp', 'SHIFT_JIS', 'Johab', 'Windows-1252', 'MacRoman']  # 他のエンコーディングも追加可能
    #  'iso-8859-1'

    for encoding in encodings_to_try:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                print(f"Trying encoding: {encoding}")
                print(f.read())  # ファイルの内容を出力
                print("------------------------------------------------")
        except UnicodeDecodeError:
            print(f"Failed to decode file with {encoding}")
            print("------------------------------------------------")

# 使用例
# file_path = "/app/data/jrdb_txt/ZED/2021/ZED211226.txt"  # 確認したいファイルのパス\
file_path = f"/app/data/jrdb_txt/{File_type}/{year}/CHA081025.txt"  # 確認したいファイルのパス
check_encoding_manually(file_path)


Trying encoding: utf-8
Failed to decode file with utf-8
------------------------------------------------
Trying encoding: euc-jp
Failed to decode file with euc-jp
------------------------------------------------
Trying encoding: SHIFT_JIS
0308310101水2008102211010415140130125 18 15 23 5631 3A3       
0308310102水2008102210221015148123126 14 22 25 6112 2A3       
0308310103水2008102210210416140130139 20 13 12 4511 2A3       
0308310104水2008102210220915142129133 18 14 18 5031 2         
0308310105水2008102210130214147126129  0 17 18 35             
0308310106木2008102311210416141132135 22 20 16 5833 405       
0308310107木2008102311210416157139133  8 15 18 4131 5OP       
0308310108木2008102311230213160160140  1  0 11 12             
0308310109水2008102210221015156140133  8  7 18 33             
0308310110水2008102210230235152134135 13 14 12 3923 310       
0308310111水2008102220110414143132123  4 11 24 3931 405       
0308310112水2008102210221044146132136 17 14 11 4223 2A3       
0308310113水200810

### CZA

In [5]:
# ディレクトリパスを指定して関数を呼び出す
check_encoding("/app/data/jrdb_txt/CZA")


File /app/data/jrdb_txt/CZA/2023/CZA231014.txt is not encoded in SHIFT_JIS. Detected encoding: CP932


In [6]:
File_type = 'CZA'
df_a = pd.read_csv(f'/app/data/jrdb_csv/{File_type}/{File_type}_2023.csv')
df_a


,調教師コード,登録抹消フラグ,登録抹消年月日,調教師名,調教師カナ,調教師名略称,所属コード,所属地域名,生年月日,初免許年,調教師コメント,コメント入力年月日,本年リーディング,本年平地成績,本年障害成績,本年特別勝数,本年重賞勝数,昨年リーディング,昨年平地成績,昨年障害成績,昨年特別勝数,昨年重賞勝数,通算平地成績,通算障害成績,データ年月日,予備
0,10014,1,19930228,中村広,ナカムラ ヒロシ,中村広,1,美浦,19150510.0,1938.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,929 930 853 5171,70 64 41 213,20231014,NaN
1,10017,1,19950228,元石正雄,モトイシ マサオ,元石正,1,美浦,19180315.0,1938.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,337 345 402 3712,14 12 15 152,20231014,NaN
2,10022,1,19920229,武平三,タケ ヘイゾウ,武平三,2,栗東,19140520.0,1938.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,587 575 614 4268,78 82 64 375,20231014,NaN
3,10023,1,19920229,田之上勲,タノウエ イサオ,田之上,2,栗東,19140630.0,1938.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,580 659 659 5345,32 32 34 161,20231014,NaN
4,10031,1,19950228,稗田敏男,ヒエダ トシオ,稗田敏,1,美浦,19181015.0,1948.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,858 757 718 5474,46 46 46 163,20231014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1303,70227,0,20201203,Ａ．マルチア,Ａ．マルチアリス,マルチ,3,仏国,19830523.0,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 0 0 1,0 0 0 0,20231014,NaN
1304,70228,0,20221202,Ｇ．ビエトリ,Ｇ．ビエトリーニ,ビエト,3,仏国,19680320.0,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 0 0 1,0 0 0 0,0.0,0.0,0 0 0 2,0 0 0 0,20231014,NaN
1305,70229,0,20221202,Ｊ．ハリント,Ｊ．ハリントン,ハリン,3,愛国,19470225.0,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 0 0 1,0 0 0 0,0.0,0.0,0 0 0 1,0 0 0 0,20231014,NaN
1306,70230,0,20221202,Ｆ．シャペ,Ｆ．シャペ,シャペ,3,仏国,19631126.0,2022.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 0 0 1,0 0 0 0,0.0,0.0,0 0 0 1,0 0 0 0,20231014,NaN


### JOA

In [7]:
File_type = 'JOA'
year = 2021
df_a = pd.read_csv(f'/app/data/jrdb_csv/{File_type}/{File_type}_{year}.csv')
df_a

df_a[df_a['日']=='a']
df_a['日'].unique()


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [8]:
# df_a[df_a['日']==2]


### KZA

In [9]:
File_type = 'KZA'
year = 2023
df_a = pd.read_csv(f'/app/data/jrdb_csv/{File_type}/{File_type}_{year}.csv')
df_a


,騎手コード,登録抹消フラグ,登録抹消年月日,騎手名,騎手カナ,騎手名略称,所属コード,所属地域名,生年月日,初免許年,...,本年重賞勝数,昨年リーディング,昨年平地成績,昨年障害成績,昨年特別勝数,昨年重賞勝数,通算平地成績,通算障害成績,データ年月日,予備
0,10007,1,19920229,増沢末夫,マスザワ スエオ,増沢末,1,NaN,19371020,1957,...,NaN,NaN,NaN,NaN,NaN,NaN,2016 1719 1457 7588,0 0 0 0,20231014,NaN
1,10011,1,19930228,徳吉一己,トクヨシ カズミ,徳吉一,1,NaN,19410201,1958,...,NaN,NaN,NaN,NaN,NaN,NaN,535 536 499 3422,1 0 1 0,20231014,NaN
2,10031,1,19930228,郷原洋行,ゴウハラ ヒロユキ,郷原洋,1,NaN,19440121,1962,...,NaN,NaN,NaN,NaN,NaN,NaN,1513 1444 1256 7685,2 0 3 3,20231014,NaN
3,10040,1,19990114,大崎昭一,オオサキ シヨウイチ,大 崎,1,NaN,19450115,1963,...,NaN,NaN,NaN,NaN,NaN,NaN,962 898 901 6085,8 4 7 19,20231014,NaN
4,10043,1,19940228,久保敏文,クボ トシフミ,久保敏,2,NaN,19431121,1963,...,NaN,NaN,NaN,NaN,NaN,NaN,555 466 529 3065,7 8 10 48,20231014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,70355,0,20230305,Ｂ．ムルザバ,Ｂ．ムルザバエフ,ムルザ,3,独国,19920917,2023,...,2.0,NaN,5 5 3 20,0 0 0 0,2.0,1.0,21 18 13 105,0 0 0 0,20231014,NaN
1214,70356,0,20230829,Ｌ．モリス,Ｌ．モリス,モリス,3,英国,19881020,2023,...,0.0,NaN,NaN,NaN,NaN,NaN,0 0 0 9,0 0 0 0,20231014,NaN
1215,70357,0,20230829,Ｍ．ヴェロン,Ｍ．ヴェロン,ヴェロ,3,仏国,19990208,2023,...,0.0,NaN,NaN,NaN,NaN,NaN,1 0 0 5,0 0 0 0,20231014,NaN
1216,70358,0,20230829,Ｒ．キング,Ｒ．キング,キング,3,英国,19900731,2023,...,0.0,NaN,NaN,NaN,NaN,NaN,1 1 0 4,0 0 0 0,20231014,NaN


## SED

In [17]:
# エンコーディングを手動で確認するためのスクリプト
def check_encoding_manually(file_path):
    encodings_to_try = ['utf-8',  'euc-jp', 'cp932']  # 他のエンコーディングも追加可能
    #  'iso-8859-1'

    for encoding in encodings_to_try:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                print(f"Trying encoding: {encoding}")
                print(f.read())  # ファイルの内容を出力
                print("------------------------------------------------")
        except UnicodeDecodeError:
            print(f"Failed to decode file with {encoding}")
            print("------------------------------------------------")

# 使用例
# file_path = "/app/data/jrdb_txt/ZED/2021/ZED211226.txt"  # 確認したいファイルのパス\
file_path = "/app/data/jrdb_txt/SED/2023/SED230105.txt"  # 確認したいファイルのパス
check_encoding_manually(file_path)


Trying encoding: utf-8
Failed to decode file with utf-8
------------------------------------------------
Trying encoding: euc-jp
Failed to decode file with euc-jp
------------------------------------------------
Trying encoding: cp932
06231101012010256220230105レディアス　　　　　　　　　　　　　12002111012A30003 　　　　　　　　　　　　　　　　　　　　　　　　　16　　　　0301129540津村明秀　　鈴木慎太郎　  36.106 41 40 -6     1                  433832HS-14.3 -5.0 -4.0 10.0シュバルツガ003353377                             4.8  38.5   4.200101007-16-151050710488428+ 61 3           480  140    0333320955
06231101022010481720230105ラディーヴァ　　　　　　　　　　　　12002111012A30003 　　　　　　　　　　　　　　　　　　　　　　　　　16　　　　0601140520原優介　　　武市康男　　 255.613 27 27 -6                        133823HS-10.3-22.0 -1.0 10.0シュバルツガ014347393                            26.1 165.1  17.400080606-10-101060410399456- 21 3                  0    0333310955
06231101032010214520230105シティオブヨーク　　　　　　　　　　12002111012A30003 　　　　　　　　　　　　　　　　　　　　　　　　　16　　　　1301150560木幡巧也　　鈴木慎太郎　  83.109 21 19 -6     2       

In [3]:
# # ディレクトリパスを指定して関数を呼び出す
# check_encoding("/app/data/jrdb_txt/ZED")
import pandas as pd

# 列を省略せずに表示
pd.set_option('display.max_columns', None)


File_type = 'SED'
year = 2023
df_a = pd.read_csv(f'/app/data/jrdb_csv/{File_type}/{File_type}_{year}.csv')
# df_a['日'].unique()
df_a


,場コード,年,回,日,Ｒ,馬番,血統登録番号,年月日,馬名,距離,芝ダ障害コード,右左,内外,馬場状態,種別,条件,記号,重量,グレード,レース名,頭数,レース名略称,着順,異常区分,タイム,斤量,騎手名,調教師名,確定単勝オッズ,確定単勝人気順位,ＩＤＭ,素点,馬場差,ペース,出遅,位置取,不利,前不利,中不利,後不利,レース,コース取り,上昇度コード,クラスコード,馬体コード,気配コード,レースペース,馬ペース,テン指数,上がり指数,ペース指数,レースＰ指数,1(2)着馬名,1(2)着タイム差,前３Ｆタイム,後３Ｆタイム,備考,予備,確定複勝オッズ下,10時単勝オッズ,10時複勝オッズ,コーナー順位１,コーナー順位２,コーナー順位３,コーナー順位４,前３Ｆ先頭差,後３Ｆ先頭差,騎手コード,調教師コード,馬体重,馬体重増減,天候コード,コース,レース脚質,単勝,複勝,本賞金,収得賞金,レースペース流れ,馬ペース流れ,４角コース取り,発走時間,改行
0,6,23,1,1,1,1,20102562,20230105,レディアス,1200,2,1,1,10,12,A3,0,3,NaN,NaN,16,NaN,3,0,1129,540,津村明秀,鈴木慎太郎,36.1,6,41.0,40.0,-6.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,38.0,3.0,2.0,H,S,-14.3,-5.0,-4.0,10.0,シュバルツガ,3.0,353.0,377.0,NaN,NaN,4.8,38.5,4.2,0.0,10.0,10.0,7.0,-16.0,-15.0,10507,10488,428,+ 6,1,NaN,3.0,NaN,480.0,140.0,0,33.0,33.0,2.0,955,NaN
1,6,23,1,1,1,2,20104817,20230105,ラディーヴァ,1200,2,1,1,10,12,A3,0,3,NaN,NaN,16,NaN,6,0,1140,520,原優介,武市康男,255.6,13,27.0,27.0,-6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,38.0,2.0,3.0,H,S,-10.3,-22.0,-1.0,10.0,シュバルツガ,14.0,347.0,393.0,NaN,NaN,26.1,165.1,17.4,0.0,8.0,6.0,6.0,-10.0,-10.0,10604,10399,456,- 2,1,NaN,3.0,NaN,NaN,0.0,0,33.0,33.0,1.0,955,NaN
2,6,23,1,1,1,3,20102145,20230105,シティオブヨーク,1200,2,1,1,10,12,A3,0,3,NaN,NaN,16,NaN,13,0,1150,560,木幡巧也,鈴木慎太郎,83.1,9,21.0,19.0,-6.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,38.0,2.0,3.0,H,S,-23.3,-15.0,-14.0,10.0,シュバルツガ,24.0,363.0,387.0,NaN,NaN,8.6,81.7,7.2,0.0,14.0,15.0,14.0,-26.0,-26.0,10582,10488,464,+12,1,NaN,4.0,NaN,NaN,0.0,0,33.0,33.0,1.0,955,NaN
3,6,23,1,1,1,4,20106714,20230105,エーデルシュタイン,1200,2,1,1,10,12,A3,0,3,NaN,NaN,16,NaN,15,0,1161,540,大野拓弥,浅野洋一郎,348.9,15,10.0,8.0,-6.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,38.0,3.0,3.0,H,S,-24.3,-25.0,-15.0,10.0,シュバルツガ,35.0,364.0,397.0,NaN,NaN,44.2,229.9,24.7,0.0,15.0,15.0,16.0,-27.0,-27.0,10516,10292,392,0,1,NaN,4.0,NaN,NaN,0.0,0,33.0,33.0,1.0,955,NaN
4,6,23,1,1,1,5,20107073,20230105,ジェイケイファイン,1200,2,1,1,10,12,A3,0,3,NaN,NaN,16,NaN,2,0,1129,560,内田博幸,奥平雅士,7.4,3,40.0,40.0,-6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,37.0,3.0,3.0,H,M,-2.3,-17.0,7.0,10.0,シュバルツガ,3.0,340.0,389.0,NaN,NaN,1.3,8.2,1.3,0.0,2.0,4.0,3.0,-3.0,-3.0,30113,10384,472,- 8,1,NaN,2.0,NaN,150.0,220.0,0,33.0,33.0,1.0,955,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38014,8,23,2,5,12,10,18101337,20231015,ヒミノフラッシュ,1800,2,1,1,32,13,16,102,1,5.0,花背ステークス,14,花背Ｓ,12,0,1548,570,和田竜二,川村禎彦,14.2,6,36.0,35.0,-4.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,29.0,2.0,2.0,M,S,-5.5,-25.2,7.2,15.6,テーオーリカ,39.0,374.0,400.0,NaN,NaN,2.5,18.8,2.6,12.0,11.0,12.0,12.0,-13.0,-13.0,10438,10317,550,- 6,1,NaN,4.0,NaN,NaN,0.0,0,32.0,23.0,4.0,1625,NaN
38015,8,23,2,5,12,11,19101736,20231015,イイネイイネイイネ,1800,2,1,1,32,13,16,102,1,5.0,花背ステークス,14,花背Ｓ,7,0,1537,520,松若風馬,小崎憲,157.5,13,37.0,37.0,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,29.0,3.0,2.0,M,S,-3.5,-25.2,7.8,15.6,テーオーリカ,28.0,367.0,396.0,NaN,NaN,23.3,58.4,9.5,5.0,5.0,5.0,4.0,-6.0,-6.0,10574,10395,450,0,1,NaN,2.0,NaN,NaN,0.0,0,32.0,23.0,2.0,1625,NaN
38016,8,23,2,5,12,12,19104907,20231015,エメヴィベール,1800,2,1,1,32,13,16,102,1,5.0,花背ステークス,14,花背Ｓ,11,0,1546,550,坂井瑠星,吉岡辰弥,7.8,5,33.0,33.0,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,29.0,3.0,7.0,M,S,-1.5,-30.2,8.2,15.6,テーオーリカ,37.0,367.0,403.0,NaN,NaN,2.2,7.6,2.7,5.0,5.0,5.0,7.0,-6.0,-8.0,10583,10485,574,+ 6,1,NaN,2.0,NaN,NaN,0.0,0,32.0,33.0,4.0,1625,NaN
38017,8,23,2,5,12,13,19104116,20231015,コパノニコルソン,1800,2,1,1,32,13,16,102,1,5.0,花背ステークス,14,花背Ｓ,8,0,1538,570,藤岡康太,宮徹,4.3,2,43.0,43.0,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,29.0,2.0,2.0,M,S,-2.5,-17.2,7.8,15.6,テーオーリカ,29.0,370.0,392.0,NaN,NaN,1.5,5.8,1.6,9.0,8.0,7.0,8.0,-9.0,-11.0,10536,10328,530,0,1,NaN,3.0,NaN,NaN,0.0,0,32.0,23.0,3.0,1625,NaN


In [1]:
import cProfile
import pstats
from io import StringIO

import cProfile

from JRDBTxtToCSV import JRDBFileConverter

def profile_function():
    File_type = "SED"
    bac_converter = JRDBFileConverter(File_type)
    bac_converter.save_to_csv(File_type, f"/app/data/jrdb_txt/{File_type}", f"/app/data/jrdb_csv/{File_type}")

# プロファイリングを実行
pr = cProfile.Profile()
pr.enable()
profile_function()
pr.disable()

# 結果を文字列として取得
s = StringIO()
sortby = 'cumulative'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()

# 結果を表示（ここでは最初の10行だけ）
print(s.getvalue().split('\n')[:10])


 52%|█████▏    | 13/25 [21:24<20:10, 100.91s/it]

## ZED

In [15]:
# # ディレクトリパスを指定して関数を呼び出す
# check_encoding("/app/data/jrdb_txt/ZED")

File_type = 'ZED'
year = 2023
df_a = pd.read_csv(f'/app/data/jrdb_csv/{File_type}/{File_type}_{year}.csv')
df_a



,場コード,年,回,日,Ｒ,年月日,発走時間,距離,芝ダ障害コード,右左,...,１着賞金,２着賞金,３着賞金,４着賞金,５着賞金,１着算入賞金,２着算入賞金,馬券発売フラグ,WIN5フラグ,予備
0,5,22,4,5,05,13201025,6220,2210,1,6,...,@,18,@,9012,45540,津村,ｾ秀,鈴木慎太郎 1,1.0,5.714
1,5,22,5,3,01,5201048,1720,2211,1,2,...,@,16,@,10014,20540,木幡,I也,武市康男 3,0.0,6.915
2,3,22,3,2,06,15201048,1720,2211,0,6,...,@,16,@,16015,40540,菊沢,齊,武市康男 2,1.0,4.415
3,4,22,2,3,06,4201048,1720,2208,0,6,...,@,18,@,15013,82540,菊沢,齊,武市康男 2,2.0,2.418
4,5,22,5,2,06,4201021,4520,2211,0,6,...,@,17,@,9012,36550,内田,詩K,鈴木慎太郎,9.0,7.913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159568,7,23,1,4,10,9171010,3620,2301,0,9,...,@,16雅,r,13015,43580,和田,メ@,谷潔 2,4.0,4.715
159569,42,21,2,A,11,5171010,3620,2102,1,0,...,@,12夢,O川特,1013,2560,田中,w,谷潔,NaN,5.000
159570,41,20,C,G,08,11171010,3620,2012,1,6,...,@,12Ｃ,P二３,1013,24560,吉村,q洋,NaN,NaN,3.000
159571,41,20,C,4,11,5171010,3620,2012,0,4,...,@,12Ｃ,Q ３,1015,45560,田中,w,NaN,NaN,2.000


In [14]:
# エンコーディングを手動で確認するためのスクリプト
def check_encoding_manually(file_path):
    encodings_to_try = ['utf-8', 'shift_jis', 'euc-jp', 'cp932']  # 他のエンコーディングも追加可能
    #  'iso-8859-1'

    for encoding in encodings_to_try:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                print(f"Trying encoding: {encoding}")
                print(f.read())  # ファイルの内容を出力
                print("------------------------------------------------")
        except UnicodeDecodeError:
            print(f"Failed to decode file with {encoding}")
            print("------------------------------------------------")

# 使用例
# file_path = "/app/data/jrdb_txt/ZED/2021/ZED211226.txt"  # 確認したいファイルのパス\
file_path = "/app/data/jrdb_txt/ZED/2008/ZED080913.txt"  # 確認したいファイルのパス
check_encoding_manually(file_path)


Trying encoding: utf-8
Failed to decode file with utf-8
------------------------------------------------
Trying encoding: shift_jis
01081504120610284320080830ヒカルジョディー　　　　　　　　　　15001112111A11023 　　　　　　　　　　　　　　　　　　　　　　　　　12　　　　0601308540秋山真一郎　中尾秀正　　  13.707 36 36 -5                        131833SS -8.9-12.5-10.3-10.3メイショウイ004361361　　　　　　　　　　　　     3.0  12.1   5.501010101  2  11044610379470   211                  0    031221    
01081304070610266820080823メジロマリアン　　　　　　　　　　　12001111011A11023 　　　　　　　　　　　　　　　　　　　　　　　　　15　　　　0201106540横山典弘　　高木登　　　  10.204 46 43 -5  3                     231732SS-15.9 -7.2 -7.5 -7.3ゲットフルマ001351355　　　　　　　　　　　　     1.9  10.8   2.600020202 -1 -11033910398472   111           240  280    022222    
01081504040610271820080830マッキーアイアン　　　　　　　　　　15001112111A11023 　　　　　　　　　　　　　　　　　　　　　　　　　12　　　　1101338540梶晃啓　　　梅田康雄　　 132.812  3  3 -5                        331822SS-20.9-36.5-16.9-10.3メイショウイ034373385　　　　　　　　　　　　    28.6  66.3  14.205050809-12 -61047010188446   213        